# Description

For each GMC mouse line hearing thresholds were used to calculate significance (Wilcoxon test) and non-parametric effect size (Cliff's delta) (see file _GMC_mannwhitneyu_results.csv_).</br>

In this notebook, the GMC mouse lines are analysed and a list of genes with significant strong effects is identified. The significance level 0.05 and the ranges for small (0.147), medium (0.33) and large (0.474) effects are considered for the thresholds determined with the three methods (manual, NN, SLR).


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline 

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Load libraries

In [ ]:
import os
import re
import warnings

import pandas  as pd
import numpy   as np
import seaborn as sns
import matplotlib.pyplot as plt

from scipy              import stats
from scipy.stats        import kendalltau
from matplotlib.patches import Patch
from matplotlib.lines   import Line2D
from matplotlib.backends.backend_pdf import PdfPages

os.environ["CUDA_VISIBLE_DEVICES"]=""
warnings.filterwarnings("ignore")

# Settings

In [ ]:
"""Set the path to the data files, for example '../data'"""
path2data = '../data'
"""Set the path to result data, for example '../results'"""
path2results = '../results'

# Description of the [GMC](https://www.mouseclinic.de/) and [ING](https://journals.plos.org/plosbiology) data

## Read data

In [ ]:
data = pd.read_csv(os.path.join(path2data, 'data4hearing_curves_analysis.csv'))
display(data.head(5))

## GMC data: basic statistics

In [ ]:
dataGMC = data[data.source == "GMC"]
display(dataGMC.head(5))

### Number of GMC mice

In [ ]:
print('GMC male mutants:   %i' % len(dataGMC[(dataGMC.sex == 'm') & (dataGMC.cohort_type == 'mut')].mouse_id.unique()))
print('GMC female mutants: %i' % len(dataGMC[(dataGMC.sex == 'f') & (dataGMC.cohort_type == 'mut')].mouse_id.unique()))
print()
print('GMC male controls:   %i' % len(dataGMC[(dataGMC.sex == 'm') & (dataGMC.cohort_type == 'con')].mouse_id.unique()))
print('GMC female controls: %i' % len(dataGMC[(dataGMC.sex == 'f') & (dataGMC.cohort_type == 'con')].mouse_id.unique()))
print()
print('Total number of GMC mutants:  %i' % len(dataGMC[dataGMC.cohort_type == 'mut'].mouse_id.unique()))
print('Total number of GMC controls: %i' % len(dataGMC[dataGMC.cohort_type == 'con'].mouse_id.unique()))
print()
print('Total number of GMC males:   %i' % len(dataGMC[dataGMC.sex == 'm'].mouse_id.unique()))
print('Total number of GMC females: %i' % len(dataGMC[dataGMC.sex == 'f'].mouse_id.unique()))

### Number of GMC genes

In [ ]:
print('Number of distinct genes: %i' % len(dataGMC[(dataGMC.cohort_type == 'mut')].gene.unique())) 

### GMC gene cohort size

In [ ]:
"""Count the number of mutants and reference controls for each gene"""
idx=0
dfGMC=pd.DataFrame(columns=['gene', 'mutants', 'controls'])
for gene in dataGMC[~dataGMC.gene.isna()].gene.unique():
    if gene:
        dfGMC.at[idx, 'gene'] = gene
        dfGMC.at[idx, 'mutants'] = dataGMC[(dataGMC.gene == gene)&(dataGMC.cohort_type == 'mut')].mouse_id.nunique()
        ref_cohorts = dataGMC[(dataGMC.gene == gene)&(dataGMC.cohort_type == 'mut')].reference_cohort.unique()
        dfGMC.at[idx, 'controls'] = dataGMC[(dataGMC.cohort_type == 'con')&(dataGMC.cohort_id.isin(ref_cohorts))].mouse_id.nunique()
        idx+=1
display(dfGMC.head(5))

In [ ]:
print('GMC mutant cohort size - median:         %i' % np.median(dfGMC.mutants))
print('GMC mutant cohort size - 5%% percentile:  %i' % np.percentile(dfGMC.mutants, 5))
print('GMC mutant cohort size - 95%% percentile: %i' % np.percentile(dfGMC.mutants, 95))
print()
print('GMC control cohort size - median:         %i' % np.median(dfGMC.controls))
print('GMC control cohort size - 5%% percentile:  %i' % np.percentile(dfGMC.controls, 5))
print('GMC control cohort size - 95%% percentile: %i' % np.percentile(dfGMC.controls, 95))

## ING data: basic statistics


In [ ]:
dataING = data[data.source == "ING"]
dataING

### Number of ING mice

In [ ]:
print('Total number of ING mutants:  %i' % len(dataING[dataING.cohort_type == 'mut'].mouse_id.unique()))
print('Total number of ING controls: %i' % len(dataING[dataING.cohort_type == 'con'].mouse_id.unique()))
print()
print('Total number of ING mice: %i' % len(dataING.mouse_id.unique()))

### Number of ING genes

In [ ]:
print('Number of distinct genes: %i' % dataING[(dataING.cohort_type == 'mut')].gene.nunique())

### ING gene cohort size

In [ ]:
"""Count the number of mutants for each gene"""
idx=0
dfING=pd.DataFrame(columns=['gene', 'mutants'])
for gene in dataING[~dataING.gene.isna()].gene.unique():
    if gene:
        dfING.at[idx, 'gene'] = gene
        dfING.at[idx, 'mutants'] = dataING[(dataING.gene == gene)&(dataING.cohort_type == 'mut')].mouse_id.nunique()
        idx+=1
display(dfING.head(5))

In [ ]:
print('ING mutant cohort size - median:         %i' % np.median(dfING.mutants))
print('ING mutant cohort size - 5%% percentile:  %i' % np.percentile(dfING.mutants, 5))
print('ING mutant cohort size - 95%% percentile: %i' % np.percentile(dfING.mutants, 95))

## GMC/ING gene overlap

In [ ]:
"""Find common genes in both datasets"""

GMCgene_set = set(dataGMC[dataGMC.cohort_type == 'mut'].gene.unique())
INGgene_set = set(dataING[dataING.cohort_type == 'mut'].gene.unique())

print(sorted(GMCgene_set.intersection(INGgene_set)))

## Available human-assigned labels per frequency and sound level (controls only)

Since we want to compare the GMC and the ING datasets, we only use control animals.

In [ ]:
"""
reduce data frame to essential columns, so we can remove redundant rows (necessary, since control mice in GMC cohorts can occur multiple times)
use controls only!
"""
data_unique = data[data.cohort_type == "con"].drop(columns = ['gene','exp_date', 'cohort_type', 'sex', 'cohort_id', 'reference_cohort','th_NN_GMCtrained', 'th_NN_INGtrained', 'th_SLR_GMCcalibrated', 'th_SLR_INGcalibrated'])
data_unique = data_unique.drop_duplicates()

data_unique.at[:,'frequency'] = ['click' if data_unique.at[idx, 'stimulation'] == 'click' else int(float(data_unique.at[idx, 'stimulation']))*1000 for idx in data_unique.index]
display(data_unique.head(2))

In [ ]:
pd.set_option("max_rows", 300)

In [ ]:
"""Aggregate GMC numbers by source, frequency, th_manual (=human-assigned threshold)"""
labels = data_unique.sort_values(['source', 'frequency', 'th_manual']).groupby(['source', 'frequency', 'th_manual'])['mouse_id'].agg(['count']).reset_index()

"""Insert temp column to make sure all positions are filled - here: 5, 95"""
fill = pd.DataFrame({"source": "GMC", "th_manual": [5,95], "click0": [0,0]} )

"""Prepare data for re-arrangement""" 
GMCc  = labels[(labels.source == 'GMC') & (labels.frequency == 'click')]
GMCc.drop(columns = 'frequency', inplace=True)
GMCc.rename(columns={'count':'click'}, inplace=True)

GMC6  = labels[(labels.source == 'GMC') & (labels.frequency ==  6000)]
GMC6.drop(columns ='frequency', inplace=True)
GMC6.rename(columns={'count':'6 kHz'}, inplace=True)

GMC12 = labels[(labels.source == 'GMC') & (labels.frequency == 12000)]
GMC12.drop(columns ='frequency', inplace=True)
GMC12.rename(columns={'count':'12 kHz'}, inplace=True)

GMC18 = labels[(labels.source == 'GMC') & (labels.frequency == 18000)]
GMC18.drop(columns ='frequency', inplace=True)
GMC18.rename(columns={'count':'18 kHz'}, inplace=True)

GMC24 = labels[(labels.source == 'GMC') & (labels.frequency == 24000)]
GMC24.drop(columns ='frequency', inplace=True)
GMC24.rename(columns={'count':'24 kHz'}, inplace=True)

GMC30 = labels[(labels.source == 'GMC') & (labels.frequency == 30000)]
GMC30.drop(columns ='frequency', inplace=True)
GMC30.rename(columns={'count':'30 kHz'}, inplace=True)

"""Merge to form new dataframe"""
result = pd.merge(fill,    GMCc, how="outer", on=["source", "th_manual"])
result = pd.merge(result,  GMC6, how="outer", on=["source", "th_manual"])
result = pd.merge(result, GMC12, how="outer", on=["source", "th_manual"])
result = pd.merge(result, GMC18, how="outer", on=["source", "th_manual"])
result = pd.merge(result, GMC24, how="outer", on=["source", "th_manual"])
result = pd.merge(result, GMC30, how="outer", on=["source", "th_manual"])
GMC_labels = result

"""Replace NaN with 0"""
GMC_labels = GMC_labels.fillna(0)

"""Drop temp column 'click0'"""
GMC_labels.drop(columns = ['source','click0'], inplace=True)

"""Sort"""
GMC_labels.sort_values(by=['th_manual'], ascending=False, ignore_index=True, inplace=True)

"""Use th_manual as index"""
GMC_labels.set_index('th_manual', inplace=True)

"""Convert to int"""
GMC_labels = GMC_labels.astype("int")

"""Max value"""
GMC_max = GMC_labels.to_numpy().max()

GMC_labels

In [ ]:
plt.figure(figsize=(16, 12))
plt.title('GMC')
sns.set(font_scale=1.9)
axGMC = sns.heatmap(GMC_labels, annot=True, fmt='d', annot_kws={"size":14}, cmap='Blues')

In [ ]:
"""Aggregate ING numbers by source, frequency, th_manual (=human-assigned threshold)"""
labels = data_unique.sort_values(['source', 'frequency', 'th_manual']).groupby(['source', 'frequency', 'th_manual'])['mouse_id'].agg(['count']).reset_index()

"""Insert temp column to make sure all positions are filled - here: 5, 95"""
fill = pd.DataFrame({"source": "ING", "th_manual": [5,95], "click0": [0,0]} )

"""Prepare data for re-arrangement"""
INGc  = labels[(labels.source == 'ING') & (labels.frequency == 'click')]
INGc.drop(columns = 'frequency', inplace=True)
INGc.rename(columns={'count':'click'}, inplace=True)

ING6  = labels[(labels.source == 'ING') & (labels.frequency ==  6000)]
ING6.drop(columns ='frequency', inplace=True)
ING6.rename(columns={'count':'6 kHz'}, inplace=True)

ING12 = labels[(labels.source == 'ING') & (labels.frequency == 12000)]
ING12.drop(columns ='frequency', inplace=True)
ING12.rename(columns={'count':'12 kHz'}, inplace=True)

ING18 = labels[(labels.source == 'ING') & (labels.frequency == 18000)]
ING18.drop(columns ='frequency', inplace=True)
ING18.rename(columns={'count':'18 kHz'}, inplace=True)

ING24 = labels[(labels.source == 'ING') & (labels.frequency == 24000)]
ING24.drop(columns ='frequency', inplace=True)
ING24.rename(columns={'count':'24 kHz'}, inplace=True)

ING30 = labels[(labels.source == 'ING') & (labels.frequency == 30000)]
ING30.drop(columns ='frequency', inplace=True)
ING30.rename(columns={'count':'30 kHz'}, inplace=True)

"""Merge to form new dataframe"""
result = pd.merge(fill,    INGc, how="outer", on=["source", "th_manual"])
result = pd.merge(result,  ING6, how="outer", on=["source", "th_manual"])
result = pd.merge(result, ING12, how="outer", on=["source", "th_manual"])
result = pd.merge(result, ING18, how="outer", on=["source", "th_manual"])
result = pd.merge(result, ING24, how="outer", on=["source", "th_manual"])
result = pd.merge(result, ING30, how="outer", on=["source", "th_manual"])
ING_labels = result

"""Replace NaN with 0"""
ING_labels = ING_labels.fillna(0)

"""Drop temp column 'click0'"""
ING_labels.drop(columns = ['source','click0'], inplace=True)

"""Sort"""
ING_labels.sort_values(by=['th_manual'], ascending=False, ignore_index=True, inplace=True)

"""Use th_manual as index"""
ING_labels.set_index('th_manual', inplace=True)

"""Convert to int"""
ING_labels = ING_labels.astype("int")

"""Max value"""
ING_max = ING_labels.to_numpy().max()

ING_labels

In [ ]:
"""Overall maximum (for use in cbar)"""
all_max = max(GMC_max, ING_max)
all_max

In [ ]:
plt.figure(figsize=(16, 12))
plt.title('ING')
sns.set(font_scale=1.9)
axING = sns.heatmap(ING_labels, annot=True, fmt='d', annot_kws={"size":14}, cmap='Blues')

In [ ]:
"""Combined plot"""
fig, axes = plt.subplots(1, 2, figsize=(24, 12), sharey=False)
# fig.suptitle('Comparison of human threshold label numbers')

GMC_controls = dataGMC[dataGMC.cohort_type == 'con'].mouse_id.nunique()
ING_controls = dataING[dataING.cohort_type == 'con'].mouse_id.nunique()

axes[0].title.set_text('GMC controls (' + str(GMC_controls) + ')')
sns.set(font_scale=1.9)
g1=sns.heatmap(GMC_labels, annot=True, fmt='d', annot_kws={"size":14}, cmap='Blues', ax=axes[0], cbar=False, vmin=0, vmax=all_max)
g1.set_yticklabels(g1.get_yticklabels(), rotation = 0)
g1.set(xlabel="stimulus", ylabel = "manual threshold [dB]")

axes[1].title.set_text('ING controls (' + str(ING_controls) + ')')
g2=sns.heatmap(ING_labels, annot=True, fmt='d', annot_kws={"size":14}, cmap='Blues', ax=axes[1], cbar=False, vmin=0, vmax=all_max)
g2.set(xlabel="stimulus", ylabel = "manual threshold [dB]")

# Line analysis

* use calculated p-values and effect sizes to compare the three methods (manual, NN, SLR) for all lines/genes
* Cliff's delta - effects according to https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.595.6157&rep=rep1&type=pdf: 
    
    small |d| > 0.147</br>
    medium |d| > 0.33</br>
    strong |d| > 0.474

In [ ]:
line_data = pd.read_csv(os.path.join(path2results, 'volcano_plots', 'GMC', 'GMC_mannwhitneyu_results.csv'))
line_data.drop(columns=[col for col in line_data.columns if 'VDA' in col], inplace=True)
line_data

In [ ]:
"""
Which genes show a significant strong effect? 
Create a pandas-data-frame to provide an overview.
"""
line_effects = line_data[['gene', 'mut_mice', 'con_mice']]

"""define thresholds for the p-value and the effect size"""
d = 0.474
pval = 0.05

"""only the click and 30 kHz stimuli are considered"""
for stimul in ['click', '30kHz']:
    for method in ['manual', 'NN_GMCtrained', 'SLR_GMCcalibrated']:
        col = stimul+'_'+method+'_up'
        line_effects[col] = [True if ((line_data.at[idx, stimul+'_th_'+method+'_pval'] < pval)&(line_data.at[idx, stimul+'_th_'+method+'_d'] > d)) else False for idx in line_effects.index]
        col = stimul+'_'+method+'_down'
        line_effects[col] = [True if ((line_data.at[idx, stimul+'_th_'+method+'_pval'] < pval)&(line_data.at[idx, stimul+'_th_'+method+'_d'] < -d)) else False for idx in line_effects.index]
            
display(line_effects.head(5))

## Utils

In [ ]:
plt.rcParams['figure.figsize'] = [24, 16]

colors = {'th_manual': '#004488'
    , 'th_manual_mut': '#004488'
    , 'th_manual_con': '#6699CC'
    , 'th_manual_ref_con': '#332288'
    , 'th_manual_all_ref_cons': '#88CCEE'
    , 'th_NN_GMCtrained': '#225522'
    , 'th_NN_GMCtrained_mut': '#225522'
    , 'th_NN_GMCtrained_con': '#5AAE61'
    , 'th_NN_GMCtrained_ref_con': '#225555'
    , 'th_NN_GMCtrained_all_ref_cons': '#CCEEFF'
    , 'th_SLR_GMCcalibrated': '#B2182B'
    , 'th_SLR_GMCcalibrated_mut': '#B2182B'
    , 'th_SLR_GMCcalibrated_con': '#D6604D'
    , 'th_SLR_GMCcalibrated_ref_con': '#882255'
    , 'th_SLR_GMCcalibrated_all_ref_cons': '#FFCCCC'
    , 'th_NN_INGtrained': '#225522'
    , 'th_NN_INGtrained_mut': '#225522'
    , 'th_NN_INGtrained_con': '#5AAE61'
    , 'th_SLR_INGcalibrated': '#B2182B'
    , 'th_SLR_INGcalibrated_mut': '#B2182B'
    , 'th_SLR_INGcalibrated_con': '#D6604D'}

std_colors = {'th_manual': '#004488'
    , 'th_NN_GMCtrained': '#CCDDAA'
    , 'th_NN_INGtrained': '#FDDBC7'
    , 'th_SLR_GMCcalibrated': '#FFCCCC'
    , 'th_SLR_INGcalibrated': '#EEEEBB'}

markers = {'con': 'o'
    , 'ref_con': 's'
    , 'mut': '^'
    , 'th_NN_GMCtrained': '^'
    , 'th_SLR_GMCcalibrated': '*'
    , 'th_manual': 'o'
    , 'th_NN_INGtrained': '^'
    , 'th_SLR_INGcalibrated': '*'}

linestyles = {'mut': 'solid'
    , 'con': 'dashed'
    , 'ref_con': 'dashdot'
    , 'th_manual': 'solid'
    , 'th_NN_GMCtrained': 'dotted'
    , 'th_SLR_GMCcalibrated': 'dashdot'
    , 'th_NN_INGtrained': 'dotted'
    , 'th_SLR_INGcalibrated': 'dashdot'}

labels = {'th_manual': 'Manually assigned thresholds'
    , 'th_NN_GMCtrained': 'Thresholds predicted by GMC trained NNs'
    , 'th_NN_INGtrained': 'Thresholds predicted by ING trained NNs'
    , 'th_SLR_GMCcalibrated': 'Thresholds estimated by GMC calibrated SLR'
    , 'th_SLR_INGcalibrated': 'Thresholds estimated by ING calibrated SLR'}

title_fontsize=20

In [ ]:
def plot_effect_size_comparison(_genes, _top_effects, _stimul, _effect=None, _idx=None, 
                                _ylim=[-1.1,1.1], _fontsize=30, _xlabel="stimulus", _ylabel="Cliff's delta", _data_source='GMC'): 
    
    """
    Method comparison: plot effect sizes for all stimuli - to show which frequencies are affected.
    """
    
    sns.set_style('white')
    
    y1 = -0.474
    y2 = 0.474
    
    noofgenes = len(_genes)
    ncols = 2
    if noofgenes > ncols:
        if noofgenes%2 == 0:
            nrows = int(noofgenes/ncols)
        else:
            nrows = int(noofgenes/ncols) + 1
    else:
        if noofgenes == 1:
            ncols = 1                   
        nrows = 1
    
    sharex = True
    if noofgenes < nrows*ncols:
        sharex = False
    
    fig, axs = plt.subplots(nrows, ncols, figsize=(ncols*12,nrows*10), sharey=True, sharex=sharex)
        
    axes = []
    if nrows == 1:
        if ncols == 1: 
            axes.append(axs)
        else:
            for ax in axs:
                axes.append(ax)
        legend_bbox = (.035, 1.24)
    else: 
        for ax1 in axs: 
            for ax2 in ax1:
                axes.append(ax2)
        legend_bbox = (.035, 1.06)
          
    lines  = []
    labels = []
        
    legend_elements = []
    
    result_file_name = _stimul
    if _effect is not None:
        result_file_name = result_file_name + '_' + _effect
    if _idx is not None:   
        result_file_name = result_file_name + '_' + str(_idx+1)
        
    for idx,ax in enumerate(axes): 
                
        xlabel = _xlabel
        ylabel = _ylabel
            
        if idx%ncols > 0:
            ylabel = None
                    
        if idx < nrows*ncols - ncols:
            xlabel = None 
            
        ax.set_ylabel(ylabel, fontsize=_fontsize)
        ax.set_xlabel(xlabel, fontsize=_fontsize, labelpad=20)
        ax.tick_params(axis='both', which='major', labelsize=_fontsize-5)
        ax.grid(zorder=0, color='lightgray')   
            
        ax.set_xticks(range(6))
        ax.set_xticklabels(["click", "6kHz", "12kHz", "18kHz", "24kHz", "30kHz"])
        
        ax.set_yticks(np.arange(-1.,1.2,0.2))
        ax.set_ylim(_ylim)
            
        if idx < noofgenes: 
            
            gene = _genes[idx]
            
            if noofgenes > 1:
                print('%i. %s' % (idx+1, gene))
            
            top_effects_man = _top_effects[["gene"] + [col for col in _top_effects.columns if 'manual' in col]]
            ax.plot(range(6), top_effects_man[top_effects_man.gene==gene].set_index("gene").T, label="manual", color='blue')
            
            top_effects_nn = _top_effects[["gene"] + [col for col in _top_effects.columns if 'NN' in col]]
            ax.plot(range(6), top_effects_nn[top_effects_nn.gene==gene].set_index("gene").T, label="NN", color='red')
            
            top_effects_slr = _top_effects[["gene"] + [col for col in _top_effects.columns if 'SLR' in col]]
            ax.plot(range(6), top_effects_slr[top_effects_slr.gene==gene].set_index("gene").T, label="SLR", color='orange')
            
            ax.axhline(y=y1, color='grey', ls='-.', lw=2, label='strong effect')
            ax.axhline(y=y2, color='grey', ls='-.', lw=2)
            
            ax.set_title("Gene: " + gene, fontsize=_fontsize, pad=15)
            
            if idx == 0:
                axLine, axLabel = ax.get_legend_handles_labels()
                lines.extend(axLine)
                labels.extend(axLabel)
            
            result_file_name = result_file_name + '_' + gene
        else:
            ax.axis('off')

    if noofgenes == 1:
        fig.legend(lines, labels, loc='lower left', bbox_to_anchor=(.15, 1.0), ncol=4,
                   borderaxespad=0, fancybox=True, fontsize=_fontsize-10)
    else:
        fig.legend(lines, labels, loc='lower left', bbox_to_anchor=(.07, 1.0), ncol=4,
                   borderaxespad=0, fancybox=True, fontsize=_fontsize-5)
        
    plt.tight_layout()
        
    fig.savefig(os.path.join(path2results, 'effect_plots', _data_source, result_file_name + '.png'), bbox_inches = 'tight', papertype = 'letter')

In [ ]:
def plot_thresholds4gene(_gene, _th_cols, _data, _colors, _markers, _linestyles, _labels,
                         _dodge=True, _fontsize=16, _markersize=7, _markerscale=1.5,
                         _cohort_type=True, _estimator=np.mean, _ci='sd', _fst_quantile=0.05, _last_quantile=0.95,
                         _mouse_id=None,
                         _legend_outside=False, _figlegend=False, _xlabel=None, _ylabel=None,
                         _fig=None, _ax=None):
    """
    Plots hearing curves for a given gene in the data set and returns the legend elements of the plot.
    """

    if _ax is None or _fig is None:
        _fig, _ax = plt.subplots()

    _data = _data.copy()

    for col in _th_cols:
        _data[col] = [100 if _data.at[_idx, col] == 999 else _data.at[_idx, col] for _idx in _data.index]

    th = re.compile('th_*')
    all_th_columns = [col for col in _data.columns if th.match(col)]

    """create dataframe with the gene mutants and the corresponding controls"""
    gene_data = _data[_data.gene == _gene]
    if _cohort_type:
        if _data.source.unique().squeeze() == 'GMC':
            gene_data = pd.concat([gene_data, _data[_data.cohort_id.isin(gene_data.reference_cohort.unique())]])
            gene_data = gene_data.replace('con', 'ref_con')
        if _data.source.unique().squeeze() == 'ING' and 'pipeline' in _data.columns:
            gene_data = pd.concat([gene_data, _data[(_data.cohort_type == 'con') & _data.pipeline.isin(gene_data.pipeline.unique())]])
        else:
            gene_data = pd.concat([gene_data, _data[_data.cohort_type == 'con']])
        gene_data = gene_data.reset_index(drop=True)

    """add threshold type column"""
    th_data = []
    for col in _th_cols:
        mouse_columns = list(gene_data.columns.drop(all_th_columns))
        mouse_columns.append(str(col))

        tmp_data = gene_data[mouse_columns]
        tmp_data = tmp_data.rename(columns={col: 'th_value'})

        for _idx in tmp_data.index:
            stim = tmp_data.at[_idx, 'stimulation']
            coh_type = tmp_data.at[_idx, 'cohort_type']
            if stim == 'click':
                if _cohort_type:
                    tmp_data.at[_idx, 'th_type'] = col+'_con_click' if coh_type=='con' else col+'_ref_con_click' if coh_type=='ref_con' else col+'_mut_click'
                else:
                    tmp_data.at[_idx, 'th_type'] = col+'_click'
            else:
                if _cohort_type:
                    tmp_data.at[_idx, 'th_type'] = col+'_con_freq' if coh_type=='con' else col+'_ref_con_freq' if coh_type=='ref_con' else col + '_mut_freq'
                else:
                    tmp_data.at[_idx, 'th_type'] = col+'_freq'
        th_data.append(tmp_data.copy())
    gene_data1 = pd.concat(th_data)

    """create legend elements"""
    legend_elements = []
    hue_order = []
    palette = {}
    markers = []
    linestyles = []

    if _data.source.unique().squeeze() == 'GMC':
        cohort_types = ['mut', 'ref_con']
    else:
        cohort_types = ['mut', 'con']

    if _cohort_type:
        for col in _th_cols:
            for coh_type in cohort_types:
                legend_elements.append(Line2D([0], [0], color=_colors[col + '_' + coh_type], marker=_markers[coh_type], linestyle=_linestyles[coh_type],
                                              ms=_markersize, label='%s%s %s (n=%d)' % (col + ' - ' if len(_th_cols)>1 else '',
                                                                                        'all controls' if coh_type=='con' else _gene+' reference controls' if coh_type=='ref_con' else _gene+' mutants',
                                                                                        'mean' if _estimator==np.mean else 'median', gene_data[gene_data.cohort_type==coh_type].mouse_id.nunique())))
                for ho in [col + '_' + coh_type + '_click', col + '_' + coh_type + '_freq']:
                    hue_order.append(ho)

        for ho in hue_order:
            if 'mut' in ho:
                for col in _th_cols:
                    if col in ho:
                        palette[ho] = _colors[col + '_mut']
                    markers.append(_markers['mut'])
                    linestyles.append(_linestyles['mut'])
            elif 'ref_con' in ho:
                for col in _th_cols:
                    if col in ho:
                        palette[ho] = _colors[col + '_ref_con']
                    markers.append(_markers['ref_con'])
                    linestyles.append(_linestyles['ref_con'])
            else:
                for col in _th_cols:
                    if col in ho:
                        palette[ho] = _colors[col + '_con']
                    markers.append(_markers['con'])
                    linestyles.append(_linestyles['con'])
    else:
        for col in _th_cols:
            if _mouse_id is None:
                if len(_th_cols)>1:
                    legend_lbl = '%s - %s' % (col, 'mean' if _estimator==np.mean else 'median')
                else:
                    legend_lbl = '%s (n=%d)' % ('mean' if _estimator==np.mean else 'median', gene_data.mouse_id.nunique())
            else:
                legend_lbl = '%s' % col
            legend_elements.append(Line2D([0], [0], color=_colors[col+'_mut'], marker=_markers[col],
                                          linestyle=_linestyles['mut'], ms=_markersize,
                                          label=legend_lbl))
            for ho in [col + '_click', col + '_freq']:
                hue_order.append(ho)
        for ho in hue_order:
            for col in _th_cols:
                if col in ho:
                    palette[ho] = _colors[col+'_mut']
                    markers.append(_markers[col])
                    linestyles.append(_linestyles['mut'])


    if _mouse_id is None:
        sns.pointplot(x='stimulation', y='th_value', data=gene_data1,
                      hue='th_type', hue_order=hue_order, dodge=_dodge, legend=False,
                      estimator=_estimator, ci=_ci, capsize=.1, errwidth=2, label=hue_order,
                      markers=markers, scale=_markerscale, linestyles=linestyles, palette=palette,
                      ax=_ax)
    elif _mouse_id in gene_data1.mouse_id.unique():
        sns.pointplot(x='stimulation', y='th_value', data=gene_data1[gene_data1.mouse_id == _mouse_id],
                      hue='th_type', hue_order=hue_order, dodge=_dodge, legend=False,
                      estimator=_estimator, ci=_ci, capsize=.1, errwidth=2, label=hue_order,
                      markers=markers, scale=_markerscale, linestyles=linestyles, palette=palette,
                      ax=_ax)

    _alpha = 0.1

    if _estimator == np.mean:
        for col in _th_cols:
            if _cohort_type:
                bounds = gene_data[gene_data.cohort_type=='con'].groupby(['stimulation', 'frequency'])[col].agg(['mean', 'std']).reset_index().sort_values(by='frequency')
            else:
                bounds = gene_data.groupby(['stimulation', 'frequency'])[col].agg(['mean', 'std']).reset_index().sort_values(by='frequency')

            x = bounds.iloc[:,0]
            collection1 = _ax.fill_between(np.arange(-0.15, 0.15, 0.01),
                                          bounds.iloc[:,2][5] - bounds.iloc[:,3][5],
                                          bounds.iloc[:,2][5] + bounds.iloc[:,3][5], alpha=_alpha,
                                          facecolor=_colors[col])
            collection1.set_zorder(0)
            collection2 = _ax.fill_between(x,
                                          bounds.iloc[:,2] - bounds.iloc[:,3],
                                          bounds.iloc[:,2] + bounds.iloc[:,3], alpha=_alpha, label='std dev con',
                                          where=[False if x[_idx] == 'click' else True for _idx in x.index],
                                          facecolor=_colors[col])
            collection2.set_zorder(0)

            if len(_th_cols)>1:
                legend_lbl="%s - %sstandard deviation" % (col, 'all controls ' if _cohort_type else '')
                if _mouse_id:
                    legend_lbl = legend_lbl + ' (n=%d)' % (gene_data[gene_data.cohort_type == 'con'].mouse_id.nunique() if _cohort_type else gene_data.mouse_id.nunique())
            else:
                legend_lbl="%sstandard deviation (n=%d)" % ('all controls ' if _cohort_type else '',
                                                            gene_data[gene_data.cohort_type == 'con'].mouse_id.nunique() if _cohort_type else gene_data.mouse_id.nunique())
            legend_elements.append(
                Patch(facecolor=_colors[col], alpha=_alpha, edgecolor=_colors[col],
                      label=legend_lbl))
    elif _estimator == np.median:
        step = _dodge/(len(_ax.get_lines())-1)
        st = -_dodge/2
        steps = []
        while st <= 0.05:
            steps.append(st)
            st += step
        j = 0
        for col in _th_cols:

            if _cohort_type:
                bounds = gene_data[_data.cohort_type=='con'].groupby(['stimulation', 'frequency'])[col].quantile((_fst_quantile,_last_quantile)).unstack().reset_index().sort_values(by='frequency')
            else:
                bounds = gene_data.groupby(['stimulation', 'frequency'])[col].quantile((_fst_quantile,_last_quantile)).unstack().reset_index().sort_values(by='frequency')

            x = bounds.iloc[:,0]

            if len(_th_cols)>1:
                legend_lbl="%s - %s[5;95] percentile range" % (col, 'all controls ' if _cohort_type else '')
                if _mouse_id:
                    legend_lbl = legend_lbl + ' (n=%d)' % (gene_data[gene_data.cohort_type == 'con'].mouse_id.nunique() if _cohort_type else gene_data.mouse_id.nunique())
            else:
                legend_lbl="%s[5;95] percentile range (n=%d)" % ('all controls ' if _cohort_type else '',
                                                                 gene_data[gene_data.cohort_type == 'con'].mouse_id.nunique() if _cohort_type else gene_data.mouse_id.nunique())

            if _cohort_type:
                for coh_type in cohort_types:
                    bounds1 = gene_data[gene_data.cohort_type==coh_type].groupby(['stimulation', 'frequency'])[col].quantile((0.25,0.75)).unstack().reset_index().sort_values(by='frequency')
                    i = 0
                    for _idx in bounds1.index:
                        x = i+steps[j]
                        if i == 0:
                            j += 1
                        ymin = bounds1.at[_idx,0.25]
                        ymax = bounds1.at[_idx,0.75]
                        vline = _ax.vlines(x, ymin=ymin, ymax=ymax, color=_colors[col + '_' + coh_type], lw=1)
                        vline.set_zorder(0)
                        hline1 = _ax.hlines(ymin, xmin=x-0.05, xmax=x+0.05, color=_colors[col + '_' + coh_type], lw=1)
                        hline2 = _ax.hlines(ymax, xmin=x-0.05, xmax=x+0.05, color=_colors[col + '_' + coh_type], lw=1)
                        i += 1
                    j += 1
            else:
                if _mouse_id is None:
                    bounds1 = gene_data.groupby(['stimulation', 'frequency'])[col].quantile((0.25,0.75)).unstack().reset_index().sort_values(by='frequency')
                    i = 0
                    for idx in bounds1.index:
                        x = i+steps[j]
                        if i == 0:
                            j += 1
                        ymin = bounds1.at[idx,0.25]
                        ymax = bounds1.at[idx,0.75]
                        vline = _ax.vlines(x, ymin=ymin, ymax=ymax, color=_colors[col], lw=1)
                        vline.set_zorder(0)
                        hline1 = _ax.hlines(ymin, xmin=x-0.05, xmax=x+0.05, color=_colors[col], lw=1)
                        hline2 = _ax.hlines(ymax, xmin=x-0.05, xmax=x+0.05, color=_colors[col], lw=1)
                        i += 1
                    j += 1

    plt.setp(_ax.lines,linewidth=1)
    if not _figlegend:
        if _legend_outside:
            _ax.legend(handles=legend_elements, loc='lower left', bbox_to_anchor= (0.0, 1.01), ncol=2,
                      borderaxespad=0, frameon=False, fontsize=_fontsize)
        else:
            _ax.legend(handles=legend_elements, loc='upper left', frameon=False, fontsize=_fontsize-8)
    else:
        _ax.legend(handles=[])

    _ax.set_ylim(0, 120)
    _ax.set_ylabel(_ylabel, fontsize=_fontsize)
    _ax.set_xlabel(_xlabel, fontsize=_fontsize, labelpad=20)
    xticklabels = []
    if _ax.get_xticklabels():
        for lb in _ax.get_xticklabels():
            txt = lb.get_text()
            if txt == 'click':
                xticklabels.append(txt)
            else:
                xticklabels.append('%ikHz' % int(float(txt)))
    _ax.set_xticklabels(xticklabels)
    _ax.set_yticks([ytick for ytick in range(20,120,20)])
    _ax.tick_params(axis='both', which='major', labelsize=_fontsize)

    return legend_elements

In [ ]:
def plot_gene_thresholds(_gene, _data, _labels, _top_effects, _nrows=2, _ncols=2,
                         _fontsize=40, _xlabel='stimulation', _ylabel='threshold [dB]',
                         _file_output_only=False, text=None):
    """
    Plots hearing curves for a given gene in the data set, taking into account the manually assessed, NN predicted and SLR estimated thresholds.
    """

    data_source = str(_data.source.unique().squeeze())

    muts = _data[_data.cohort_type == 'mut']

    cols = ['th_manual', 'th_NN_'+data_source+'trained', 'th_SLR_'+data_source+'calibrated']
    titles = {}
    for col in cols:
        titles[col] = '%s\n(%s)' % (_labels[col], col)

    y1 = -0.474
    y2 = 0.474

    markersize = 10
    markerscale = 2

    nrows = _nrows
    ncols = _ncols
    xlabel = _xlabel

    fig1, axs1 = plt.subplots(nrows, ncols, figsize=(ncols*15,nrows*13), sharex=True, constrained_layout=True)

    nrow = -1
    ncol = 0
    for _idx,col in enumerate(cols):

        xlabel = _xlabel
        ylabel = _ylabel

        ncol = _idx%ncols

        if ncol == 0:
            nrow += 1
        else:
            ylabel = None

        if nrow == 0 and nrows > 1:
            xlabel = None

        if nrows == 1:
            ax = axs1[ncol]
        else:
            ax = axs1[nrow, ncol]

        plot_thresholds4gene(_gene, [col], _data, colors, markers, linestyles, labels,
                             _estimator=np.median, _ci=None, _dodge=0.1,
                             _fontsize=_fontsize, _markersize=markersize, _markerscale=markerscale,
                             _xlabel=xlabel, _ylabel=ylabel, _fig=fig1, _ax=ax)

        ax.set_title(titles[col], fontsize=_fontsize+5, y=1.01)
        if text and _idx < len(text):
            ax.text(-0.3, 3., text[_idx], fontsize=_fontsize+10, fontweight='bold')
        ax.grid(zorder=0, color='lightgray')

        ncol += 1

    if nrows == 1:
        ax = axs1[ncol]
    else:
        ax = axs1[nrow, ncol]


    top_effects_man = _top_effects[["gene"] + [col for col in _top_effects.columns if 'manual' in col]]
    ax.scatter([0], top_effects_man[top_effects_man.gene==_gene][['gene', 'click_th_manual_d']].set_index("gene").T,
               s=plt.rcParams['lines.markersize'] ** 3.2, marker=markers['mut'], color=colors['th_manual_mut'])
    ax.plot(range(1,6), top_effects_man[top_effects_man.gene==_gene][top_effects_man.columns.drop(['click_th_manual_d'])].set_index("gene").T,
            label="manual", color=colors['th_manual_mut'])

    top_effects_nn = _top_effects[["gene"] + [col for col in _top_effects.columns if 'NN' in col]]
    ax.scatter([0], top_effects_nn[top_effects_nn.gene==_gene][['gene', 'click_th_NN_GMCtrained_d']].set_index("gene").T,
               s=plt.rcParams['lines.markersize'] ** 3.2, marker=markers['mut'], color=colors['th_NN_GMCtrained_mut'])
    ax.plot(range(1, 6),
            top_effects_nn[top_effects_nn.gene==_gene][top_effects_nn.columns.drop(['click_th_NN_GMCtrained_d'])].set_index("gene").T,
            label="NN", color=colors['th_NN_GMCtrained_mut'])

    top_effects_slr = _top_effects[["gene"] + [col for col in _top_effects.columns if 'SLR' in col]]
    ax.scatter([0], top_effects_slr[top_effects_slr.gene==_gene][['gene', 'click_th_SLR_GMCcalibrated_d']].set_index("gene").T,
               s=plt.rcParams['lines.markersize'] ** 3.2, marker=markers['mut'], color=colors['th_SLR_GMCcalibrated_mut'])
    ax.plot(range(1, 6),
            top_effects_slr[top_effects_slr.gene==_gene][top_effects_slr.columns.drop(['click_th_SLR_GMCcalibrated_d'])].set_index("gene").T,
            label="SLR", color=colors['th_SLR_GMCcalibrated_mut'])

    ax.axhline(y=y1, color='grey', ls='-.', lw=2, label='strong effect')
    ax.axhline(y=0.0, color='grey', ls='dashed', lw=2, label='strong effect')
    ax.axhline(y=y2, color='grey', ls='-.', lw=2)

    ax.set_title("Mutant vs. control effect sizes", fontsize=_fontsize+5, y=1.01)
    if text and _idx < len(text):
        ax.text(-0.3, -0.95, text[-1], fontsize=_fontsize+10, fontweight='bold')
    ax.grid(zorder=0, color='lightgray')
    ax.set_ylim(-1.0, 1.0)
    # axs1[nrow, ncol].set_ylabel(ylabel, fontsize=_fontsize)
    ax.set_xlabel(xlabel, fontsize=_fontsize, labelpad=20)
    ax.tick_params(axis='both', which='major', labelsize=_fontsize)

    legend_elements = [Line2D([0], [0], color=colors['th_manual_mut'], linestyle=linestyles['mut'], label='manual'),
                       Line2D([0], [0], color=colors['th_NN_GMCtrained_mut'], linestyle=linestyles['mut'],label='NN'),
                       Line2D([0], [0], color=colors['th_SLR_GMCcalibrated_mut'], linestyle=linestyles['mut'],label='SLR'),
                       Line2D([0], [0], color='grey', linestyle='-.',label='strong effect')]

    ax.legend(handles=legend_elements, loc='upper left', frameon=False, fontsize=_fontsize-8, ncol=2)

    fig1.tight_layout()

    #         pdf.savefig(fig1, bbox_inches = 'tight', papertype = 'letter')
    fig1.savefig(os.path.join('../results', _gene + '_median_hearing_and_curves_effect_sizes.png'))
    if _file_output_only:
        plt.close()

## Click

List genes with significant and strong effects: **p<0.05 & |d| > 0.474**.

### Effect based on manually assessed thresholds

In [ ]:
"""Effect based on manually assessed thresholds"""

top_effects_click_manual_up = line_data[line_data.gene.isin(line_effects[line_effects.click_manual_up].gene)].sort_values(by=['click_th_manual_d'], ascending=False)[['gene', 'click_th_manual_d']]
print('----------------------------------------------------------------')
print('Genes with significant p-values and strong positive effects (%i):' % top_effects_click_manual_up.gene.nunique())
print('----------------------------------------------------------------')
display(top_effects_click_manual_up)

top_effects_click_manual_down = line_data[line_data.gene.isin(line_effects[line_effects.click_manual_down].gene)].sort_values(by=['click_th_manual_d'], ascending=True)[['gene', 'click_th_manual_d']]
print('----------------------------------------------------------------')
print('Genes with significant p-values and strong negative effects (%i):' % top_effects_click_manual_down.gene.nunique())
print('----------------------------------------------------------------')
display(top_effects_click_manual_down)

### Effect based on NN predicted thresholds

In [ ]:
"""Effect based on NN predicted thresholds"""

top_effects_click_NN_up = line_data[line_data.gene.isin(line_effects[line_effects.click_NN_GMCtrained_up].gene)].sort_values(by=['click_th_NN_GMCtrained_d'], ascending=False)[['gene', 'click_th_NN_GMCtrained_d']]
print('----------------------------------------------------------------')
print('Genes with significant p-values and strong positive effects (%i):' % top_effects_click_NN_up.gene.nunique())
print('----------------------------------------------------------------')
display(top_effects_click_NN_up)

top_effects_click_NN_down = line_data[line_data.gene.isin(line_effects[line_effects.click_NN_GMCtrained_down].gene)].sort_values(by=['click_th_NN_GMCtrained_d'], ascending=True)[['gene', 'click_th_NN_GMCtrained_d']]
print('----------------------------------------------------------------')
print('Genes with significant p-values and strong negative effects (%i):' % top_effects_click_NN_down.gene.nunique())
print('----------------------------------------------------------------')
display(top_effects_click_NN_down)

### Effect based on SLR estimated thresholds

In [ ]:
"""Effect based on SLR estimated thresholds"""

top_effects_click_SLR_up = line_data[line_data.gene.isin(line_effects[line_effects.click_SLR_GMCcalibrated_up].gene)].sort_values(by=['click_th_SLR_GMCcalibrated_d'], ascending=False)[['gene', 'click_th_SLR_GMCcalibrated_d']]
print('----------------------------------------------------------------')
print('Genes with significant p-values and strong positive effects (%i):' % top_effects_click_SLR_up.gene.nunique())
print('----------------------------------------------------------------')
display(top_effects_click_SLR_up)

top_effects_click_SLR_down = line_data[line_data.gene.isin(line_effects[line_effects.click_SLR_GMCcalibrated_down].gene)].sort_values(by=['click_th_SLR_GMCcalibrated_d'], ascending=True)[['gene', 'click_th_SLR_GMCcalibrated_d']]
print('----------------------------------------------------------------')
print('Genes with significant p-values and strong negative effects (%i):' % top_effects_click_SLR_down.gene.nunique())
print('----------------------------------------------------------------')
display(top_effects_click_SLR_down)

### Complete list of genes

In [ ]:
top_effects_click_up = line_data[line_data.gene.isin(line_effects[line_effects.click_manual_up | line_effects.click_NN_GMCtrained_up | line_effects.click_SLR_GMCcalibrated_up].gene)].sort_values(by=['click_th_manual_d'], ascending=False)[['gene', 'click_th_manual_d', 'click_th_NN_GMCtrained_d', 'click_th_SLR_GMCcalibrated_d']]
print('----------------------------------------------------------------')
print('Genes with significant p-values and strong positive effects (%i):\n  manual: %i genes\n  NN:\t  %i genes\n  SLR:\t  %i genes\n  total:  %i genes' % 
      (top_effects_click_up.gene.nunique(), top_effects_click_manual_up.gene.nunique(), top_effects_click_NN_up.gene.nunique(), top_effects_click_SLR_up.gene.nunique(), 
       len(np.unique(np.union1d(np.unique(np.union1d(top_effects_click_manual_up.gene.unique(), top_effects_click_NN_up.gene.unique())), top_effects_click_SLR_up.gene.unique())))))
print('----------------------------------------------------------------')
display(top_effects_click_up)

In [ ]:
top_effects_click_down = line_data[line_data.gene.isin(line_effects[line_effects.click_manual_down | line_effects.click_NN_GMCtrained_down | line_effects.click_SLR_GMCcalibrated_down].gene)].sort_values(by=['click_th_manual_d'], ascending=True)[['gene', 'click_th_manual_d', 'click_th_NN_GMCtrained_d', 'click_th_SLR_GMCcalibrated_d']]
print('----------------------------------------------------------------')
print('Genes with significant p-values and strong negative effects (%i):\n  manual: %i genes\n  NN:\t  %i genes\n  SLR:\t  %i genes\n  total:  %i genes' % 
      (top_effects_click_down.gene.nunique(), top_effects_click_manual_down.gene.nunique(), top_effects_click_NN_down.gene.nunique(), top_effects_click_SLR_down.gene.nunique(), 
       len(np.unique(np.union1d(np.unique(np.union1d(top_effects_click_manual_down.gene.unique(), top_effects_click_NN_down.gene.unique())), top_effects_click_SLR_down.gene.unique())))))
print('----------------------------------------------------------------')
display(top_effects_click_down)

In [ ]:
top_effects_click = line_data[line_data.gene.isin(top_effects_click_up.gene) | line_data.gene.isin(top_effects_click_down.gene)][["gene"] + [col for col in line_data.columns if '_d' in col]]
top_effects_click['effect'] = ['up' if (top_effects_click.at[idx,'gene'] in top_effects_click_up.gene.values) else 'down' for idx in top_effects_click.index]
top_effects_click.sort_values(by=['click_th_manual_d'], ascending=False, inplace=True)
top_effects_click = top_effects_click[['gene', 'effect'] + [col for col in line_data.columns if '_d' in col]]
print('----------------------------------------------------------------')
print('Click - strong effects: %i genes\n  up:\t%i genes\n  down:\t%i genes' % 
      (top_effects_click.gene.nunique(), top_effects_click[top_effects_click.effect == 'up'].gene.nunique(), top_effects_click[top_effects_click.effect == 'down'].gene.nunique()))
print('----------------------------------------------------------------')
display(top_effects_click)

In [ ]:
"""Concatenate top click genes"""
top_click_genes = pd.concat([pd.Series(top_effects_click_up.gene).reset_index().drop(columns='index').rename(columns={"gene": "genes_up_combined"}),
                             pd.Series(top_effects_click_manual_up.gene).reset_index().drop(columns='index').rename(columns={"gene": "genes_up_manual"}), 
                             pd.Series(top_effects_click_NN_up.gene).reset_index().drop(columns='index').rename(columns={"gene": "genes_up_NN"}),  
                             pd.Series(top_effects_click_SLR_up.gene).reset_index().drop(columns='index').rename(columns={"gene": "genes_up_SLR"}),
                             pd.Series(top_effects_click_down.gene).reset_index().drop(columns='index').rename(columns={"gene": "genes_down_combined"}),
                             pd.Series(top_effects_click_manual_down.gene).reset_index().drop(columns='index').rename(columns={"gene": "genes_down_manual"}), 
                             pd.Series(top_effects_click_NN_down.gene).reset_index().drop(columns='index').rename(columns={"gene": "genes_down_NN"}),  
                             pd.Series(top_effects_click_SLR_down.gene).reset_index().drop(columns='index').rename(columns={"gene": "genes_down_SLR"})], 
                            axis = 1)

top_click_genes.replace(np.nan, '', regex=True)

### Plot comparisons for all genes with strong positive effects

In [ ]:
for idx, gene in enumerate(top_effects_click_up.sort_values(by=['click_th_manual_d'], ascending=False).gene[:1]):
    plot_effect_size_comparison(_genes=[gene], _top_effects=top_effects_click, _stimul='click', _effect='up', _idx=idx, 
                                _fontsize=30, _xlabel="stimulation", _ylabel="Cliff's delta")

### Plot comparisons for all genes with strong negative effects

In [ ]:
for idx, gene in enumerate(top_effects_click_down.sort_values(by=['click_th_manual_d'], ascending=True).gene[:1]):
    plot_effect_size_comparison(_genes=[gene], _top_effects=top_effects_click, _stimul='click', _effect='down', _idx=idx, 
                                _fontsize=30, _xlabel="stimulation", _ylabel="Cliff's delta")

## 30 kHz

List genes with significant and strong effects: p<0.05 & |d| > 0.474.

### Effect based on manually assessed thresholds

In [ ]:
"""Effect based on manually assessed thresholds"""

top_effects_30kHz_manual_up = line_data[line_data.gene.isin(line_effects[line_effects['30kHz_manual_up']].gene)].sort_values(by=['30kHz_th_manual_d'], ascending=False)[['gene', '30kHz_th_manual_d']]
print('----------------------------------------------------------------')
print('Genes with significant p-values and strong positive effects (%i):' % top_effects_30kHz_manual_up.gene.nunique())
print('----------------------------------------------------------------')
display(top_effects_30kHz_manual_up)

top_effects_30kHz_manual_down = line_data[line_data.gene.isin(line_effects[line_effects['30kHz_manual_down']].gene)].sort_values(by=['30kHz_th_manual_d'], ascending=True)[['gene', '30kHz_th_manual_d']]
print('----------------------------------------------------------------')
print('Genes with significant p-values and strong negative effects (%i):' % top_effects_30kHz_manual_down.gene.nunique())
print('----------------------------------------------------------------')
display(top_effects_30kHz_manual_down)

### Effect based on NN predicted thresholds

In [ ]:
"""Effect based on NN predicted thresholds"""

top_effects_30kHz_NN_up = line_data[line_data.gene.isin(line_effects[line_effects['30kHz_NN_GMCtrained_up']].gene)].sort_values(by=['30kHz_th_NN_GMCtrained_d'], ascending=False)[['gene', '30kHz_th_NN_GMCtrained_d']]
print('----------------------------------------------------------------')
print('Genes with significant p-values and strong positive effects (%i):' % top_effects_30kHz_NN_up.gene.nunique())
print('----------------------------------------------------------------')
display(top_effects_30kHz_NN_up)

top_effects_30kHz_NN_down = line_data[line_data.gene.isin(line_effects[line_effects['30kHz_NN_GMCtrained_down']].gene)].sort_values(by=['30kHz_th_NN_GMCtrained_d'], ascending=True)[['gene', '30kHz_th_NN_GMCtrained_d']]
print('----------------------------------------------------------------')
print('Genes with significant p-values and strong negative effects (%i):' % top_effects_30kHz_NN_down.gene.nunique())
print('----------------------------------------------------------------')
display(top_effects_30kHz_NN_down)

### Effect based on SLR estimated thresholds

In [ ]:
"""Effect based on SLR estimated thresholds"""

top_effects_30kHz_SLR_up = line_data[line_data.gene.isin(line_effects[line_effects['30kHz_SLR_GMCcalibrated_up']].gene)].sort_values(by=['30kHz_th_SLR_GMCcalibrated_d'], ascending=False)[['gene', '30kHz_th_SLR_GMCcalibrated_d']]
print('----------------------------------------------------------------')
print('Genes with significant p-values and strong positive effects (%i):' % top_effects_30kHz_SLR_up.gene.nunique())
print('----------------------------------------------------------------')
display(top_effects_30kHz_SLR_up)

top_effects_30kHz_SLR_down = line_data[line_data.gene.isin(line_effects[line_effects['30kHz_SLR_GMCcalibrated_down']].gene)].sort_values(by=['30kHz_th_SLR_GMCcalibrated_d'], ascending=True)[['gene', '30kHz_th_SLR_GMCcalibrated_d']]
print('----------------------------------------------------------------')
print('Genes with significant p-values and strong negative effects (%i):' % top_effects_30kHz_SLR_down.gene.nunique())
print('----------------------------------------------------------------')
display(top_effects_30kHz_SLR_down)

### Complete list of genes

In [ ]:
top_effects_30kHz_up = line_data[line_data.gene.isin(line_effects[line_effects['30kHz_manual_up'] | line_effects['30kHz_NN_GMCtrained_up'] | line_effects['30kHz_SLR_GMCcalibrated_up']].gene)].sort_values(by=['30kHz_th_manual_d'], ascending=False)[['gene', '30kHz_th_manual_d', '30kHz_th_NN_GMCtrained_d', '30kHz_th_SLR_GMCcalibrated_d']]
print('----------------------------------------------------------------')
print('Genes with significant p-values and strong positive effects (%i):\n  manual: %i genes\n  NN:\t  %i genes\n  SLR:\t  %i genes\n  total:  %i genes' % 
      (top_effects_30kHz_up.gene.nunique(), top_effects_30kHz_manual_up.gene.nunique(), top_effects_30kHz_NN_up.gene.nunique(), top_effects_30kHz_SLR_up.gene.nunique(), 
       len(np.unique(np.union1d(np.unique(np.union1d(top_effects_30kHz_manual_up.gene.unique(), top_effects_30kHz_NN_up.gene.unique())), top_effects_30kHz_SLR_up.gene.unique())))))
print('----------------------------------------------------------------')
display(top_effects_30kHz_up)

In [ ]:
top_effects_30kHz_down = line_data[line_data.gene.isin(line_effects[line_effects['30kHz_manual_down'] | line_effects['30kHz_NN_GMCtrained_down'] | line_effects['30kHz_SLR_GMCcalibrated_down']].gene)].sort_values(by=['30kHz_th_manual_d'], ascending=False)[['gene', '30kHz_th_manual_d', '30kHz_th_NN_GMCtrained_d', '30kHz_th_SLR_GMCcalibrated_d']]
print('----------------------------------------------------------------')
print('Genes with significant p-values and strong positive effects (%i):\n  manual: %i genes\n  NN:\t  %i genes\n  SLR:\t  %i genes\n  total:  %i genes' % 
      (top_effects_30kHz_down.gene.nunique(), top_effects_30kHz_manual_down.gene.nunique(), top_effects_30kHz_NN_down.gene.nunique(), top_effects_30kHz_SLR_down.gene.nunique(), 
       len(np.unique(np.union1d(np.unique(np.union1d(top_effects_30kHz_manual_down.gene.unique(), top_effects_30kHz_NN_down.gene.unique())), top_effects_30kHz_SLR_down.gene.unique())))))
print('----------------------------------------------------------------')
display(top_effects_30kHz_down)

In [ ]:
top_effects_30kHz = line_data[line_data.gene.isin(top_effects_30kHz_up.gene) | line_data.gene.isin(top_effects_30kHz_down.gene)][["gene"] + [col for col in line_data.columns if '_d' in col]]
top_effects_30kHz['effect'] = ['up' if (top_effects_30kHz.at[idx,'gene'] in top_effects_30kHz_up.gene.values) else 'down' for idx in top_effects_30kHz.index]
top_effects_30kHz.sort_values(by=['30kHz_th_manual_d'], ascending=False, inplace=True)
top_effects_30kHz = top_effects_30kHz[['gene', 'effect'] + [col for col in line_data.columns if '_d' in col]]
print('----------------------------------------------------------------')
print('30kHz - strong effects: %i genes\n  up:\t%i genes\n  down:\t%i genes' % 
      (top_effects_30kHz.gene.nunique(), top_effects_30kHz[top_effects_30kHz.effect == 'up'].gene.nunique(), top_effects_30kHz[top_effects_30kHz.effect == 'down'].gene.nunique()))
print('----------------------------------------------------------------')
display(top_effects_30kHz)

In [ ]:
"""Concatenate top click genes"""
top_30kHz_genes = pd.concat([pd.Series(top_effects_30kHz_up.gene).reset_index().drop(columns='index').rename(columns={"gene": "genes_up_combined"}),
                             pd.Series(top_effects_30kHz_manual_up.gene).reset_index().drop(columns='index').rename(columns={"gene": "genes_up_manual"}), 
                             pd.Series(top_effects_30kHz_NN_up.gene).reset_index().drop(columns='index').rename(columns={"gene": "genes_up_NN"}),  
                             pd.Series(top_effects_30kHz_SLR_up.gene).reset_index().drop(columns='index').rename(columns={"gene": "genes_up_SLR"}),
                             pd.Series(top_effects_30kHz_down.gene).reset_index().drop(columns='index').rename(columns={"gene": "genes_down_combined"}),
                             pd.Series(top_effects_30kHz_manual_down.gene).reset_index().drop(columns='index').rename(columns={"gene": "genes_down_manual"}), 
                             pd.Series(top_effects_30kHz_NN_down.gene).reset_index().drop(columns='index').rename(columns={"gene": "genes_down_NN"}),  
                             pd.Series(top_effects_30kHz_SLR_down.gene).reset_index().drop(columns='index').rename(columns={"gene": "genes_down_SLR"})], 
                            axis = 1)

top_30kHz_genes.replace(np.nan, '', regex=True)

### Plot comparisons for all genes with strong positive effects

In [ ]:
for idx, gene in enumerate(top_effects_30kHz_up.sort_values(by=['30kHz_th_manual_d'], ascending=False).gene[:1]):
    plot_effect_size_comparison(_genes=[gene], _top_effects=top_effects_30kHz, _stimul='30kHz', _effect='up', _idx=idx, 
                                _fontsize=30, _xlabel="stimulation", _ylabel="Cliff's delta")

### Plot comparisons for all genes with strong negative effects

In [ ]:
for idx, gene in enumerate(top_effects_30kHz_down.sort_values(by=['30kHz_th_manual_d'], ascending=True).gene[:1]):
    plot_effect_size_comparison(_genes=[gene], _top_effects=top_effects_30kHz, _stimul='30kHz', _effect='down', _idx=idx, 
                                _fontsize=30, _xlabel="stimulation", _ylabel="Cliff's delta")

## Various comparison plots

### Alkbh6 and Mgat1

Plot comparisons for these genes with strong effects:
* Alkbh6: 30kHz negative
* Mgat1: 30kHz positive


In [ ]:
path2results = '../results'
plot_effect_size_comparison(_genes=['Alkbh6', 'Mgat1'], _top_effects=top_effects_30kHz, _stimul='30kHz', _effect=None, _idx=None, 
                            _fontsize=30, _xlabel="stimulation", _ylabel="Cliff's delta")

### Hunk, Plekha1, Alkbh6, Ngdn

Plot comparisons for these genes with strong positive effects.

In [ ]:
plot_effect_size_comparison(_genes=['Hunk', 'Plekha1', 'Alkbh6', 'Ngdn'], _top_effects=top_effects_click, _stimul='click', _effect=None, _idx=None, 
                            _fontsize=30, _xlabel="stimulation", _ylabel="Cliff's delta")

### Vps13c, Rabgap1, Ttll12, Hdac1, Adprm, Kansl1l

Plot comparisons for these genes with strong effects:
* Vps13c: click positive
* Rabgap1: click positive
* Ttll12: click and 30kHz positive
* Hdac1: click positive
* Adprm: click and 30kHz positive
* Kansl1: click positive

In [ ]:
plot_effect_size_comparison(_genes=['Vps13c', 'Rabgap1', 'Ttll12', 'Hdac1', 'Adprm', 'Kansl1l'], _top_effects=top_effects_click, _stimul='click', _effect=None, _idx=None, 
                            _fontsize=30, _xlabel="stimulation", _ylabel="Cliff's delta")

### Ngdn, Gpatch2l and Gdi2 

Plot comparisons for these genes with strong effects:
* Ngdn: click and 30kHz positive
* Gpatch2l: click positive
* Gdi2: click positive

In [ ]:
plot_effect_size_comparison(_genes=['Ngdn', 'Gpatch2l', 'Gdi2'], _top_effects=top_effects_click, _stimul='click', _effect=None, _idx=None, 
                            _fontsize=30, _xlabel="stimulation", _ylabel="Cliff's delta")

## Comparison of mutants vs. control effect sizes

In [ ]:
dataGMC = dataGMC.astype({"stimulation": str})
display(dataGMC.head(2))

curvesGMC = pd.read_csv(os.path.join(path2data, 'GMC', 'GMC_abr_curves.csv'))
display(curvesGMC.head(2))

### Hunk

In [ ]:
plt.rcParams['figure.figsize'] = [30, 26]
plot_gene_thresholds('Hunk', dataGMC, _labels=labels, _top_effects=top_effects_click, _nrows=1, _ncols=4,
                     _fontsize=40, _xlabel='stimulation', _ylabel='threshold [dB]',
                     _file_output_only=False, text=['A', 'B', 'C', 'D'])

### Ngdn

In [ ]:
plt.rcParams['figure.figsize'] = [30, 26]
plot_gene_thresholds('Ngdn', dataGMC, _labels=labels, _top_effects=top_effects_click, _nrows=1, _ncols=4,
                     _fontsize=40, _xlabel='stimulation', _ylabel='threshold [dB]',
                     _file_output_only=False, text=['E', 'F', 'G', 'H'])